In [5]:
#pre-processesing
import numpy as np
path = '/Users/tanushree/Desktop/ML PROJECT/data/'
ac_class = []
for i in range(1,20):
    if i < 10:
        ac_class.append('a0'+str(i))
    else:
        ac_class.append('a'+str(i))

ac_person = []
for i in range(1,9):
        ac_person.append('p'+str(i))

ac_file = []
for i in range(1,61):
    if i < 10:
        ac_file.append('s0' + str(i))
    else:
        ac_file.append('s' + str(i))

print(ac_class)
print(ac_person)
print(ac_file)

###################normal class#########################
normal = []
normal_class = [i for i in range(19) if i not in [4,5,11,17,18]]
for i in range(14):
    for j in range(8):
        for z in range(60):
            print(ac_class[normal_class[i]] +'/'+ ac_person[j]+'/'+ ac_file[z]+ '.txt')
            data = np.loadtxt(path + ac_class[normal_class[i]] +'/'+ 
                              ac_person[j]+'/'+ ac_file[z] + '.txt',delimiter= ',')
            normal.append(data)
    print(len(normal))
normal = np.array(normal)
print(normal.shape)
np.save("/Users/tanushree/Desktop/ML PROJECT/normal.npy", normal)

###################abnormal class#########################
abnormal_class = [4,5,11,17,18]
abnormal = []
for i in range(5):
    for j in range(8):
        for z in range(60):
            print(ac_class[abnormal_class[i]] +'/'+ ac_person[j]+'/'+ ac_file[z]+ '.txt')
            data = np.loadtxt(path + ac_class[abnormal_class[i]] +'/'+ 
                              ac_person[j]+'/'+ ac_file[z] + '.txt',delimiter= ',')
            abnormal.append(data) 
    print(len(abnormal))
abnormal = np.array(abnormal)
print(abnormal.shape)
np.save("/Users/tanushree/Desktop/ML PROJECT/abnormal.npy", abnormal)



['a01', 'a02', 'a03', 'a04', 'a05', 'a06', 'a07', 'a08', 'a09', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19']
['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8']
['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's39', 's40', 's41', 's42', 's43', 's44', 's45', 's46', 's47', 's48', 's49', 's50', 's51', 's52', 's53', 's54', 's55', 's56', 's57', 's58', 's59', 's60']
a01/p1/s01.txt
a01/p1/s02.txt
a01/p1/s03.txt
a01/p1/s04.txt
a01/p1/s05.txt
a01/p1/s06.txt
a01/p1/s07.txt
a01/p1/s08.txt
a01/p1/s09.txt
a01/p1/s10.txt
a01/p1/s11.txt
a01/p1/s12.txt
a01/p1/s13.txt
a01/p1/s14.txt
a01/p1/s15.txt
a01/p1/s16.txt
a01/p1/s17.txt
a01/p1/s18.txt
a01/p1/s19.txt
a01/p1/s20.txt
a01/p1/s21.txt
a01/p1/s22.txt
a01/p1/s23.txt
a01/p1/s24.txt
a01/p1/s25.txt
a01/p1/s26.txt
a01/p1

In [2]:
#transformation
import numpy as np
import os
import scipy.io as scio
import random
from random import choice
from scipy.signal import savgol_filter, medfilt
from sklearn import preprocessing

global a,dim_DA
a = 125
dim_DA = 45

def inter_data(hr, window=11):
    N = window
    time3 = savgol_filter(hr, window_length=N, polyorder=2)
    return time3

def noised(signal):
    SNR = 5
    noise = np.random.randn(signal.shape[0], signal.shape[1])
    noise = noise - np.mean(noise)
    signal_power = np.linalg.norm(signal) ** 2 / signal.size
    noise_variance = signal_power / np.power(10, (SNR / 10))
    noise = (np.sqrt(noise_variance) / np.std(noise)) * noise
    signal_noise = noise + signal
    return signal_noise

def negated(signal):
    return signal * -1

def opposite_time(signal):
    return signal[::-1,:]

def permuted(signal):
    listA = [0,1,2,3,4]
    random.shuffle(listA)
    sig = signal[listA[0]*25:listA[0]*25+25]
    for i in range(1,len(listA)):
        sig = np.vstack((sig,signal[listA[i]*25:listA[i]*25+25]))
    return sig

def scale(signal):
    sc = [0.5, 2, 1.5, 0.8]
    s = choice(sc)
    return signal * s

def time_warp(signal):
    for i in range(signal.shape[1]):
        signal[:,i] = inter_data(signal[:,i],11)
    return signal

def regular_mm(data):
    dim = dim_DA * a
    data = data.reshape(data.shape[0], dim)
    min_max_scaler = preprocessing.MinMaxScaler()
    data = min_max_scaler.fit_transform(data)
    data = data.reshape(data.shape[0], a, dim_DA)
    return data

def transformation(dataX):
    data_no = np.zeros((dataX.shape[0],dataX.shape[1],dataX.shape[2]))
    data_ne = np.zeros((dataX.shape[0],dataX.shape[1],dataX.shape[2]))
    data_op = np.zeros((dataX.shape[0], dataX.shape[1], dataX.shape[2]))
    data_pe = np.zeros((dataX.shape[0], dataX.shape[1], dataX.shape[2]))
    data_sc = np.zeros((dataX.shape[0], dataX.shape[1], dataX.shape[2]))
    data_ti = np.zeros((dataX.shape[0], dataX.shape[1], dataX.shape[2]))

    for i in range(dataX.shape[0]):
        data_no[i] = noised(dataX[i].copy())
        data_ne[i] = negated(dataX[i].copy())
        data_op[i] = opposite_time(dataX[i].copy())
        data_pe[i] = permuted(dataX[i].copy())
        data_sc[i] = scale(dataX[i].copy())
        data_ti[i] = time_warp(dataX[i].copy())

    #####################Normalization###########################
    data_raw = regular_mm(dataX)
    data_no = regular_mm(data_no)
    data_ne = regular_mm(data_ne)
    data_op= regular_mm(data_op)
    data_pe = regular_mm(data_pe)
    data_sc = regular_mm(data_sc)
    data_ti = regular_mm(data_ti)

    data_raw = np.reshape(data_raw, data_raw.shape + (1,))
    data_no = np.reshape(data_no, data_no.shape + (1,))
    data_ne = np.reshape(data_ne, data_ne.shape + (1,))
    data_op = np.reshape(data_op, data_op.shape + (1,))
    data_pe = np.reshape(data_pe, data_pe.shape + (1,))
    data_sc = np.reshape(data_sc, data_sc.shape + (1,))
    data_ti = np.reshape(data_ti, data_ti.shape + (1,))

    return data_raw,data_no,data_ne,data_op,data_pe,data_sc,data_ti

def shuffle(normal_s_raw,normal_s_no,normal_s_ne,normal_s_op,normal_s_pe,normal_s_sc,normal_s_ti):
    path = '/Users/tanushree/Desktop/ML PROJECT/dataset_normalize_together/'
    ######################shuffle data#########################
    listA = [l for l in range(normal_s_no.shape[0])]
    random.shuffle(listA)
    listB = [p for p in range(normal_s_no.shape[0])]

    dataset_raw = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_no = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_ne = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_op = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_pe = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_sc = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])
    dataset_ti = np.zeros(
        [normal_s_no.shape[0], normal_s_no.shape[1], normal_s_no.shape[2], normal_s_no.shape[3]])

    for w, r in zip(listA, listB):
        dataset_raw[r, :, :, :] = normal_s_raw[w, :, :, :]
        dataset_no[r, :, :, :] = normal_s_no[w, :, :, :]
        dataset_ne[r, :, :, :] = normal_s_ne[w, :, :, :]
        dataset_op[r, :, :, :] = normal_s_op[w, :, :, :]
        dataset_pe[r, :, :, :] = normal_s_pe[w, :, :, :]
        dataset_sc[r, :, :, :] = normal_s_sc[w, :, :, :]
        dataset_ti[r, :, :, :] = normal_s_ti[w, :, :, :]

    print('shuffle done')

    X_train_raw = dataset_raw[:int(dataset_raw.shape[0] * 0.6), :, :, :]
    X_test_raw = dataset_raw[int(dataset_raw.shape[0] * 0.6):, :, :, :]

    X_train_no = dataset_no[:int(dataset_no.shape[0] * 0.6), :, :, :]
    X_test_no = dataset_no[int(dataset_no.shape[0] * 0.6):, :, :, :]

    X_train_ne = dataset_ne[:int(dataset_ne.shape[0] * 0.6), :, :, :]
    X_test_ne = dataset_ne[int(dataset_ne.shape[0] * 0.6):, :, :, :]

    X_train_op = dataset_op[:int(dataset_op.shape[0] * 0.6), :, :, :]
    X_test_op = dataset_op[int(dataset_op.shape[0] * 0.6):, :, :, :]

    X_train_pe = dataset_pe[:int(dataset_pe.shape[0] * 0.6), :, :, :]
    X_test_pe = dataset_pe[int(dataset_pe.shape[0] * 0.6):, :, :, :]

    X_train_sc = dataset_sc[:int(dataset_sc.shape[0] * 0.6), :, :, :]
    X_test_sc = dataset_sc[int(dataset_sc.shape[0] * 0.6):, :, :, :]

    X_train_ti = dataset_ti[:int(dataset_ti.shape[0] * 0.6), :, :, :]
    X_test_ti = dataset_ti[int(dataset_ti.shape[0] * 0.6):, :, :, :]

    print('save preparing')

    np.save(path + "data_raw_train.npy", X_train_raw)
    np.save(path + "data_no_train.npy", X_train_no)
    np.save(path + "data_ne_train.npy", X_train_ne)
    np.save(path + "data_op_train.npy", X_train_op)
    np.save(path + "data_pe_train.npy", X_train_pe)
    np.save(path + "data_sc_train.npy", X_train_sc)
    np.save(path + "data_ti_train.npy", X_train_ti)

    np.save(path + "data_raw_test.npy", X_test_raw)
    np.save(path + "data_no_test.npy", X_test_no)
    np.save(path + "data_ne_test.npy", X_test_ne)
    np.save(path + "data_op_test.npy", X_test_op)
    np.save(path + "data_pe_test.npy", X_test_pe)
    np.save(path + "data_sc_test.npy", X_test_sc)
    np.save(path + "data_ti_test.npy", X_test_ti)


if __name__ == '__main__':
    normal = np.load("/Users/tanushree/Desktop/ML PROJECT/normal.npy")
    abnormal = np.load("/Users/tanushree/Desktop/ML PROJECT/abnormal.npy")
    number = normal.shape[0]
    dataX = np.vstack((normal,abnormal))

    data_raw, data_no, data_ne, data_op, data_pe, data_sc, data_ti = transformation(dataX)

    data_raw_n = data_raw[:number]
    data_no_n = data_no[:number]
    data_ne_n = data_ne[:number]
    data_op_n = data_op[:number]
    data_pe_n = data_pe[:number]
    data_sc_n = data_sc[:number]
    data_ti_n = data_ti[:number]

    data_raw_a = data_raw[number:]
    data_no_a = data_no[number:]
    data_ne_a = data_ne[number:]
    data_op_a = data_op[number:]
    data_pe_a = data_pe[number:]
    data_sc_a = data_sc[number:]
    data_ti_a = data_ti[number:]

    ####################################save normal data######################
    shuffle( data_raw_n,data_no_n,data_ne_n,data_op_n,data_pe_n,data_sc_n,data_ti_n)

    ####################################save abnormal data######################
    path = '/Users/tanushree/Desktop/ML PROJECT/dataset_normalize_together/'
    np.save(path + "data_raw_abnormal.npy", data_raw_a)
    np.save(path + "data_no_abnormal.npy", data_no_a)
    np.save(path + "data_ne_abnormal.npy", data_ne_a)
    np.save(path + "data_op_abnormal.npy", data_op_a)
    np.save(path + "data_pe_abnormal.npy", data_pe_a)
    np.save(path + "data_sc_abnormal.npy", data_sc_a)
    np.save(path + "data_ti_abnormal.npy", data_ti_a)

shuffle done
save preparing


In [ ]:
#training
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras import optimizers, regularizers
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
from keras.losses import categorical_crossentropy
from keras.backend import permute_dimensions
from keras.activations import softmax, relu
from keras.initializers import RandomUniform
from keras import backend as K
from keras.utils import np_utils
from keras.optimizers import Adam, SGD
import tensorflow.python.keras.backend as KTF
import numpy as np
import scipy.io as scio
import random
import math
import os
from sklearn import metrics
import argparse
from pyimagesearch.learning_rate_schedulers import StepDecay
from pyimagesearch.learning_rate_schedulers import PolynomialDecay
import csv
# import the necessary packages
import numpy as np
import cv2

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
KTF.set_session(sess)

# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-e", "--epochs", type=int, default=100,help="# of epochs to train for")
ap.add_argument("-e", "--transformation", type=int, default=7,help="# of transformation number")
ap.add_argument("-e", "--input_dim_x", type=int, default=125,help="# of input_dim_x")
ap.add_argument("-e", "--input_dim_y", type=int, default=45,help="# of input_dim_y")
ap.add_argument("-e", "--global_mem_dim", type=int, default=500,help="# of global_mem_dim")
ap.add_argument("-e", "--local_mem_dim", type=int, default=500,help="# of local_mem_dim")
ap.add_argument("-e", "--filter_size0", type=int, default=1,help="# of filter_size0")
ap.add_argument("-e", "--filter_size1", type=int, default=32,help="# of filter_size1")
ap.add_argument("-e", "--filter_size2", type=int, default=64,help="# of filter_size2")
ap.add_argument("-e", "--filter_size3", type=int, default=128,help="# of filter_size3")
ap.add_argument("-e", "--lambda1", type=float, default=1.0,help="# of lambda1")
ap.add_argument("-e", "--lambda2", type=float, default=0.0002,help="# of lambda2")
ap.add_argument("-e", "--batch", type=int, default=16,help="# of batch size")
ap.add_argument('--data_path', type = str, default = '/Users/tanushree/Desktop/ML PROJECT/dataset_normalize_together/',
                   help='path to load data')
ap.add_argument('--model_path', type = str, default = '/Users/tanushree/Desktop/ML PROJECT/model_train/',
                   help='path to save model')

args = vars(ap.parse_args())
epochs = args["epochs"]
trans = args["transformation"]
input_dim_x = args["input_dim_x"]
input_dim_y = args["input_dim_y"]
global_mem_dim = args["global_mem_dim"]
local_mem_dim = args["local_mem_dim"]
filter_size0 = args["filter_size0"]
filter_size1 = args["filter_size1"]
filter_size2 = args["filter_size2"]
filter_size3 = args["filter_size3"]
l1 = args["lambda1"]
l2 = args["lambda2"]
batch = args["batch"]
path = args["data_path"]
model_path = args["model_path"]

class Memory_global(Layer):
    def __init__(self, mem_dim, fea_dim,**kwargs):
        # C: dimension of vector z
        # M: size of the memory
        super(Memory_global, self).__init__(**kwargs)
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.kernel_regularizer = None
        self.std = 1. / math.sqrt(self.fea_dim)

    def build(self, input_shape):
        # M x C
        self.weight = self.add_weight(name='kernel',shape=(self.fea_dim,self.mem_dim),
                                      initializer=RandomUniform(-self.std, self.std),
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)
        super(Memory_global, self).build(input_shape)

    def get_config(self):
        config = {'mem_dim': self.mem_dim,'fea_dim': self.fea_dim}
        base_config = super(Memory_global, self).get_config()
        print(base_config)
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        x1_size = K.int_shape(inputs)[1]
        x2_size = K.int_shape(inputs)[2]
        channel = K.int_shape(inputs)[3]

        inputs = K.reshape(inputs,[-1,channel])

        distance = K.dot(inputs, K.transpose(self.weight))  # Fea x Mem^T, (TxC) x (CxM) = TxM
        att_weight = softmax(distance,axis=1)

        output = K.dot(att_weight, self.weight)  # AttWeight x Mem^T^T = AW x Mem, (TxM) x (MxC) = TxC
        output =K.reshape(output,[-1, x1_size, x2_size, channel])
        att = tf.reshape(att_weight, [-1,  x1_size* x2_size* self.fea_dim])
        att =K.mean(-att * K.log(att), axis=-1, keepdims=True)
        return [output, att]

    def compute_output_shape(self, input_shape):
       #assert isinstance(input_shape, list)
        #return [(input_shape[0],input_shape[1],input_shape[2],input_shape[3]),(input_shape[0],input_shape[1],input_shape[2],self.mem_dim)]
        return [(input_shape[0], input_shape[1], input_shape[2], input_shape[3]),
                (input_shape[0],1)]

class Memory_local(Layer):
    def __init__(self, mem_dim, fea_dim,**kwargs):
        # C: dimension of vector z
        # M: size of the memory
        super(Memory_local, self).__init__(**kwargs)
        self.mem_dim = mem_dim
        self.fea_dim = fea_dim
        self.kernel_regularizer = None
        self.std = 1. / math.sqrt(self.fea_dim)

    def build(self, input_shape):
        # M x C
        self.weight = self.add_weight(name='kernel_local',shape=(self.fea_dim,self.mem_dim),
                                      initializer=RandomUniform(-self.std, self.std),
                                      regularizer=self.kernel_regularizer,
                                      trainable=True)
        super(Memory_local, self).build(input_shape)

    def get_config(self):
        config = {'mem_dim': self.mem_dim,'fea_dim': self.fea_dim}
        base_config = super(Memory_local, self).get_config()
        print(base_config)
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        x1_size = K.int_shape(inputs)[1]
        x2_size = K.int_shape(inputs)[2]
        channel = K.int_shape(inputs)[3]

        inputs = K.reshape(inputs,[-1,channel])

        distance = K.dot(inputs, K.transpose(self.weight))  # Fea x Mem^T, (TxC) x (CxM) = TxM
        att_weight = softmax(distance,axis=1)

        output = K.dot(att_weight, self.weight)  # AttWeight x Mem^T^T = AW x Mem, (TxM) x (MxC) = TxC
        output =K.reshape(output,[-1, x1_size, x2_size, channel])
        att = tf.reshape(att_weight, [-1,  x1_size* x2_size* self.fea_dim])
        att = K.mean(-att * K.log(att), axis=-1, keepdims=True)
        return [output, att]

    def compute_output_shape(self, input_shape):
       #assert isinstance(input_shape, list)
        #return [(input_shape[0],input_shape[1],input_shape[2],input_shape[3]),(input_shape[0],input_shape[1],input_shape[2],self.mem_dim)]
        return [(input_shape[0], input_shape[1], input_shape[2], input_shape[3]),
                (input_shape[0],1)]

def slice(x, index):
    return x[:, index, :, :, :]

def mse_compute(x,index):
    encoder = x[0][:,index,:,:,:]
    decoder = x[1]
    return K.mean(K.square(encoder-decoder))

def conlstm_auto():
    print("training start")
    #############################encoder########################
    encoder_input1 = Input(shape=(trans, input_dim_x,input_dim_y, 1),name="encoder_input_no")
    x1 = TimeDistributed(ZeroPadding2D(padding =((3,0),(3,0)) , data_format="channels_last", name='zero'), name='T0')(encoder_input1)
    x1 =TimeDistributed(Conv2D(filter_size1, (4, 4), activation='relu', padding='same', data_format="channels_last",name = 'conv1'),name='T1')(x1)
    x1 = TimeDistributed(MaxPooling2D((2,2), padding='same', name='pool1'), name='T2')(x1)
    x1 = TimeDistributed(Conv2D(filter_size2, (4, 4), activation='relu', padding='same', data_format="channels_last",name = 'conv2'),name='T3')(x1)
    encoded1 = TimeDistributed(MaxPooling2D((2,2), padding='same', name='pool2'), name='T4')(x1)

    x1_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 0},name='L1')(encoded1)
    x2_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 1},name='L2')(encoded1)
    x3_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 2},name='L3')(encoded1)
    x4_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 3},name='L4')(encoded1)
    x5_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 4},name='L5')(encoded1)
    x6_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 5},name='L6')(encoded1)
    x7_ = Lambda(slice, output_shape=(32, 12, filter_size2), arguments={'index': 6},name='L7')(encoded1)

    #############################self supervision########################
    inp1 = Input(shape = (32, 12, filter_size2), name='global_class')
    predict = Conv2D(1, (4, 4), padding='same', activation='sigmoid', data_format="channels_last")(inp1)
    predict = Flatten()(predict)
    predict = Dense(128, activation='relu')(predict)
    predict = Dropout(0.5)(predict)
    predict = Dense(7, activation='softmax')(predict)
    model_class = Model(inputs=inp1, outputs=predict,name = 'model_class')

    g1 = model_class(x1_)
    g2 = model_class(x2_)
    g3 = model_class(x3_)
    g4 = model_class(x4_)
    g5 = model_class(x5_)
    g6 = model_class(x6_)
    g7 = model_class(x7_)

    #############################global memory########################
    inp = Input(shape = (32, 12, filter_size2), name='global_input')
    memory_output, att_weight = Memory_local(mem_dim=filter_size2, fea_dim=global_mem_dim)(inp)
    model_global = Model(inputs=inp, outputs=[memory_output,att_weight],name='global_memory')

    memory_output_g1, att_weight_g1 = model_global(x1_)
    memory_output_g2, att_weight_g2 = model_global(x2_)
    memory_output_g3, att_weight_g3 = model_global(x3_)
    memory_output_g4, att_weight_g4 = model_global(x4_)
    memory_output_g5, att_weight_g5 = model_global(x5_)
    memory_output_g6, att_weight_g6 = model_global(x6_)
    memory_output_g7, att_weight_g7 = model_global(x7_)

    memory_global_sparse = Add()(
        [att_weight_g1, att_weight_g2, att_weight_g3, att_weight_g4, att_weight_g5, att_weight_g6, att_weight_g7])

    #############################local memory########################
    memory_output1, att_weight1 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_1')(x1_)
    memory_output2, att_weight2 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_2')(x2_)
    memory_output3, att_weight3 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_3')(x3_)
    memory_output4, att_weight4 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_4')(x4_)
    memory_output5, att_weight5 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_5')(x5_)
    memory_output6, att_weight6 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_6')(x6_)
    memory_output7, att_weight7 = Memory_local(mem_dim=filter_size2, fea_dim=local_mem_dim,name='memory_local_7')(x7_)

    memory_local_sparse = Add()(
        [att_weight1, att_weight2, att_weight3, att_weight4, att_weight5, att_weight6, att_weight7])

    #############################adaptive fusion########################
    c = Input(shape=(1,))
    c1 = Dense(2*trans)(c)
    c1 = BatchNormalization(momentum=0.93)(c1,training=False)
    c1 = Activation('sigmoid')(c1)

    m_out1 = Lambda(lambda x: x[:,0], name='c1')(c1)
    m_out2 = Lambda(lambda x: x[:,1], name='c2')(c1)
    local_weight1 = Multiply(name='weight_l1')([memory_output1, m_out1])
    global_weight1 = Multiply(name='weight_g1')([memory_output_g1, m_out2])
    final_1 = Add()([local_weight1,global_weight1])

    m_out3 = Lambda(lambda x: x[:,2], name='c3')(c1)
    m_out4 = Lambda(lambda x: x[:,3], name='c4')(c1)
    local_weight2 = Multiply(name='weight_l2')([memory_output2, m_out3])
    global_weight2 = Multiply(name='weight_g2')([memory_output_g2, m_out4])
    final_2 = Add()([local_weight2,global_weight2])

    m_out5 = Lambda(lambda x: x[:,4], name='c5')(c1)
    m_out6 = Lambda(lambda x: x[:,5], name='c6')(c1)
    local_weight3 = Multiply(name='weight_l3')([memory_output3, m_out5])
    global_weight3 = Multiply(name='weight_g3')([memory_output_g3, m_out6])
    final_3 = Add()([local_weight3,global_weight3])

    m_out7 = Lambda(lambda x: x[:,6], name='c7')(c1)
    m_out8 = Lambda(lambda x: x[:,7], name='c8')(c1)
    local_weight4 = Multiply(name='weight_l4')([memory_output4, m_out7])
    global_weight4 = Multiply(name='weight_g4')([memory_output_g4, m_out8])
    final_4 = Add()([local_weight4,global_weight4])

    m_out9 = Lambda(lambda x: x[:,8], name='c9')(c1)
    m_out10 = Lambda(lambda x: x[:,9], name='c10')(c1)
    local_weight5 = Multiply(name='weight_l5')([memory_output5, m_out9])
    global_weight5 = Multiply(name='weight_g5')([memory_output_g5, m_out10])
    final_5 = Add()([local_weight5,global_weight5])

    m_out11 = Lambda(lambda x: x[:,10], name='c11')(c1)
    m_out12 = Lambda(lambda x: x[:,11], name='c12')(c1)
    local_weight6 = Multiply(name='weight_l6')([memory_output1, m_out11])
    global_weight6 = Multiply(name='weight_g6')([memory_output_g1, m_out12])
    final_6 = Add()([local_weight6,global_weight6])

    m_out13 = Lambda(lambda x: x[:,12], name='c13')(c1)
    m_out14 = Lambda(lambda x: x[:,13], name='c14')(c1)
    local_weight7 = Multiply(name='weight_l7')([memory_output1, m_out13])
    global_weight7 = Multiply(name='weight_g7')([memory_output_g1, m_out14])
    final_7 = Add()([local_weight7,global_weight7])

    memory_output_raw = Concatenate(axis=-1,name='concat_local1_uni')([x1_, final_1])
    memory_output_no = Concatenate(axis=-1,name='concat_local2_uni')([x2_, final_2])
    memory_output_ne = Concatenate(axis=-1,name='concat_local3_uni')([x3_, final_3])
    memory_output_op = Concatenate(axis=-1,name='concat_local4_uni')([x4_, final_4])
    memory_output_pe = Concatenate(axis=-1,name='concat_local5_uni')([x5_, final_5])
    memory_output_sc = Concatenate(axis=-1,name='concat_local6_uni')([x6_, final_6])
    memory_output_ti = Concatenate(axis=-1,name='concat_local7_uni')([x7_, final_7])

    #############################decoder########################

    xx1_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose1_1')(memory_output_raw)
    xx1_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose1_2')(xx1_l)
    xx1_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose1_3')(xx1_l)
    xx1_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose1_4')(xx1_l)
    decoder1_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose1_5')(xx1_l)

    xx2_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose2_1')(memory_output_no)
    xx2_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose2_2')(xx2_l)
    xx2_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose2_3')(xx2_l)
    xx2_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose2_4')(xx2_l)
    decoder2_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose2_5')(xx2_l)

    xx3_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose3_1')(memory_output_ne)
    xx3_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose3_2')(xx3_l)
    xx3_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose3_3')(xx3_l)
    xx3_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose3_4')(xx3_l)
    decoder3_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose3_5')(xx3_l)

    xx4_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose4_1')(memory_output_op)
    xx4_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose4_2')(xx4_l)
    xx4_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose4_3')(xx4_l)
    xx4_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose4_4')(xx4_l)
    decoder4_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose4_5')(xx4_l)

    xx5_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose5_1')(memory_output_pe)
    xx5_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose5_2')(xx5_l)
    xx5_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose5_3')(xx5_l)
    xx5_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose5_4')(xx5_l)
    decoder5_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose5_5')(xx5_l)

    xx6_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose6_1')(memory_output_sc)
    xx6_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose6_2')(xx6_l)
    xx6_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose6_3')(xx6_l)
    xx6_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose6_4')(xx6_l)
    decoder6_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose6_5')(xx6_l)

    xx7_l = Conv2DTranspose(filter_size3, (4, 4), padding='same', activation='relu',name='transpose7_1')(memory_output_ti)
    xx7_l = Conv2DTranspose(filter_size2, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose7_2')(xx7_l)
    xx7_l = Conv2DTranspose(filter_size1, (4, 4), padding='same', strides=(2, 2), activation='relu',name='transpose7_3')(xx7_l)
    xx7_l = Conv2DTranspose(filter_size0, (4, 4), padding='same', activation='sigmoid',name='transpose7_4')(xx7_l)
    decoder7_l = Cropping2D(cropping=((3, 0), (3, 0)), data_format="channels_last",name='transpose7_5')(xx7_l)

    mse_loss1_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 0},name='mse1')([encoder_input1, decoder1_l])
    mse_loss2_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 1},name='mse2')([encoder_input1, decoder2_l])
    mse_loss3_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 2},name='mse3')([encoder_input1, decoder3_l])
    mse_loss4_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 3},name='mse4')([encoder_input1, decoder4_l])
    mse_loss5_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 4},name='mse5')([encoder_input1, decoder5_l])
    mse_loss6_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 5},name='mse6')([encoder_input1, decoder6_l])
    mse_loss7_l = Lambda(mse_compute, output_shape=(1,), arguments={'index': 6},name='mse7')([encoder_input1, decoder7_l])

    mse_loss = Add()([mse_loss1_l, mse_loss2_l, mse_loss3_l, mse_loss4_l, mse_loss5_l, mse_loss6_l, mse_loss7_l])

    sparse_loss = Add()([memory_global_sparse, memory_local_sparse])

    COMPOSITE_ED = Model(inputs=[encoder_input1,c],
                         outputs=[mse_loss,sparse_loss,g1,g2,g3,g4,g5,g6,g7])

    COMPOSITE_ED.compile(loss=[lambda y_true, y_pred: y_pred, lambda y_true, y_pred: y_pred,'categorical_crossentropy',
                               'categorical_crossentropy','categorical_crossentropy','categorical_crossentropy',
                               'categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                         loss_weights=[1, l2, l1, l1, l1, l1, l1, l1, l1], optimizer='Adam')
    return COMPOSITE_ED

if __name__ == '__main__':
    model = conlstm_auto()

    X_train_raw = np.load(path + "data_raw_train.npy")
    X_train_no = np.load(path + "data_no_train.npy")
    X_train_ne = np.load(path + "data_ne_train.npy")
    X_train_op = np.load(path + "data_op_train.npy")
    X_train_pe = np.load(path + "data_pe_train.npy")
    X_train_sc = np.load(path + "data_sc_train.npy")
    X_train_ti = np.load(path + "data_ti_train.npy")

    X_train = np.concatenate((X_train_raw,X_train_no,X_train_ne,X_train_op,X_train_pe,X_train_sc,X_train_ti),axis=-1)
    X_train = X_train.transpose(0,-1,1,2)
    X_train = np.reshape(X_train, X_train.shape + (1,))

    #############################model train#########################

    if not os.path.exists(model_path):
        os.makedirs(model_path)

    filepath = model_path+'model_best_weight.h5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True,
                                 save_best_only=True, mode='min')

    dataY1 = np.zeros((X_train.shape[0], 1))
    initial_c = np.zeros((X_train.shape[0], 1))

    label = []
    for i in range(trans):
        for j in range(X_train.shape[0]):
            label.append(i)
    label = np.array(label)
    y_classes = np_utils.to_categorical(label)
    n = X_train.shape[0]
    y_1 = y_classes[:n]
    y_2= y_classes[n:n*2]
    y_3= y_classes[n*2:n*3]
    y_4= y_classes[n*3:n*4]
    y_5= y_classes[n*4:n*5]
    y_6= y_classes[n*5:n*6]
    y_7= y_classes[n*6:n*7]

    history = model.fit([X_train,initial_c],
                        [dataY1, dataY1,y_1,y_2,y_3,y_4,y_5,y_6,y_7], epochs=epochs, batch_size=batch, callbacks=checkpoint, validation_split=0.2)

    [predict_label8, predict_label9, predict_label1, predict_label2,predict_label3,predict_label4,predict_label5, predict_label6, predict_label7] = model.predict([X_train,initial_c], batch_size=batch, verbose=1)

    class_loss = predict_label1[:, 0] + predict_label2[:, 1] + predict_label3[:, 2] + predict_label4[:,
                                                                                      3] + predict_label5[:,
                                                                                           4] + predict_label6[:,
                                                                                                5] + predict_label7[:,
                                                                                                     6]
    np.savetxt(model_path + 'train_normal_loss_sparse.csv', predict_label9,delimiter=',')
    np.savetxt(model_path + 'train_normal_loss_class.csv', class_loss, delimiter=',')
    np.savetxt(model_path + 'train_normal_loss_sum_mse.csv', predict_label8, delimiter=',')

    #############################model test#########################
    X_test_raw = np.load(path + "data_raw_test.npy")
    X_test_no = np.load(path + "data_no_test.npy")
    X_test_ne = np.load(path + "data_ne_test.npy")
    X_test_op = np.load(path + "data_op_test.npy")
    X_test_pe = np.load(path + "data_pe_test.npy")
    X_test_sc = np.load(path + "data_sc_test.npy")
    X_test_ti = np.load(path + "data_ti_test.npy")

    X_test = np.concatenate((X_test_raw, X_test_no,X_test_ne,X_test_op,X_test_pe,X_test_sc,X_test_ti),axis=-1)
    X_test = X_test.transpose(0,-1,1,2)
    X_test = np.reshape(X_test, X_test.shape + (1,))
    initial_c_test = np.zeros((X_test.shape[0], 1))

    [predict_label8,predict_label9, predict_label1, predict_label2,predict_label3,predict_label4,predict_label5, predict_label6, predict_label7] = model.predict([X_test,initial_c_test], batch_size=batch, verbose=1)

    class_loss = predict_label1[:, 0] + predict_label2[:, 1] + predict_label3[:, 2] + predict_label4[:,
                                                                                      3] + predict_label5[:,
                                                                                           4] + predict_label6[:,
                                                                                                5] + predict_label7[:,
                                                                                                     6]

    np.savetxt(model_path+ 'normal_loss_sparse.csv', predict_label9, delimiter=',')
    np.savetxt(model_path+'normal_loss_class.csv', class_loss, delimiter=',')
    np.savetxt(model_path+ 'normal_loss_sum_mse.csv', predict_label8, delimiter=',')

    abnormal_s_raw = np.load(path + "data_raw_abnormal.npy")
    abnormal_s_no = np.load(path + "data_no_abnormal.npy")
    abnormal_s_ne = np.load(path + "data_ne_abnormal.npy")
    abnormal_s_op = np.load(path + "data_op_abnormal.npy")
    abnormal_s_pe = np.load(path + "data_pe_abnormal.npy")
    abnormal_s_sc = np.load(path + "data_sc_abnormal.npy")
    abnormal_s_ti = np.load(path + "data_ti_abnormal.npy")

    abnormal = np.concatenate((abnormal_s_raw, abnormal_s_no,abnormal_s_ne,abnormal_s_op,abnormal_s_pe,abnormal_s_sc,abnormal_s_ti), axis=-1)
    abnormal = abnormal.transpose(0, -1, 1, 2)
    abnormal = np.reshape(abnormal, abnormal.shape + (1,))
    initial_c_ab = np.zeros((abnormal.shape[0], 1))

    [predict_label8, predict_label9,predict_label1, predict_label2,predict_label3,predict_label4,predict_label5, predict_label6, predict_label7] = model.predict([abnormal,initial_c_ab], batch_size=batch,verbose=1)

    class_loss = predict_label1[:, 0] + predict_label2[:, 1] + predict_label3[:, 2] + predict_label4[:,
                                                                                      3] + predict_label5[:,
                                                                                           4] + predict_label6[:,
                                                                                                5] + predict_label7[:,
                                                                                                     6]

    np.savetxt(model_path + 'abnormal_loss_sparse.csv', predict_label9,delimiter=',')
    np.savetxt(model_path + 'abnormal_loss_class.csv', class_loss, delimiter=',')
    np.savetxt(model_path + 'abnormal_loss_sum_mse.csv', predict_label8, delimiter=',')


In [ ]:
#evaluate
import csv
import numpy as np

csv_file = csv.reader(open(r'train_normal_loss_sum_mse.csv', 'r'))
data1=[]
for row in csv_file:
    data1.append(row)
b_t = []
for i in range(len(data1)):
    b_t.append(float(data1[i][0]))
bb_t=np.array(b_t)
thre = np.percentile(bb_t, 98)  #Percentile

csv_file = csv.reader(open(r'normal_loss_sum_mse', 'r'))
data=[]
for row in csv_file:
    data.append(row)
b = []
for i in range(len(data)):
    b.append(float(data[i][0]))
bb=np.array(b)
print(bb.shape)

normal_true = bb[bb[:] < thre]
print('normal_sum :' ,bb.shape[0],'normal_true',normal_true.shape[0])

csv_file = csv.reader(open(r'abnormal_loss_sum_mse', 'r'))
data=[]
for row in csv_file:
    data.append(row)
b = []
for i in range(len(data)):
    b.append(float(data[i][0]))
bb1=np.array(b)
print(bb1.shape)

abnormal_true = bb1[bb1[:] >= thre]
print('abnormal_sum :' ,bb1.shape[0],'normal_true',abnormal_true.shape[0])

normal_true = normal_true.shape[0]
fatigue_true = abnormal_true.shape[0]
recerr2 = bb.shape[0]
recerr3 = bb1.shape[0]

acc = (normal_true + fatigue_true) / (recerr2 + recerr3)
precision_n = normal_true / (normal_true + recerr3 - fatigue_true)
precision_a = fatigue_true / (fatigue_true + recerr2 - normal_true)
pre_avg = (precision_a + precision_n) / 2
recall_n = normal_true / recerr2
recall_a = fatigue_true / recerr3
recall_avg = (recall_a + recall_n) / 2
F1 = 2 * (pre_avg * recall_avg) / (pre_avg + recall_avg)

from prettytable import PrettyTable
x = PrettyTable(["acc", "pre_normal", "pre_abn", "pre_avg", "recall_normal", "recall_abn", "recall_avg", "F1"])
x.add_row([acc, precision_n, precision_a, pre_avg, recall_n, recall_a, recall_avg, F1])
print(x)